<center style="font-size:48px;">Modeling</center>

In [10]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import IsolationForest

locations = pd.read_csv('../CheckPoints/Locations_Clean.csv', parse_dates=['Timestamp'], index_col=0)
locations.head()

,Timestamp,id,lat,long,LastName,FirstName,CurrentEmploymentType,CurrentEmploymentTitle,day,hour,minute,second
0,2014-01-06 06:28:01,35,36.076223,24.874643,Vasco-Pais,Willem,Executive,Environmental Safety Advisor,6,6,28,1
1,2014-01-06 06:28:01,35,36.076223,24.874643,Vasco-Pais,Willem,Executive,Environmental Safety Advisor,6,6,28,1
2,2014-01-06 06:28:03,35,36.076211,24.874443,Vasco-Pais,Willem,Executive,Environmental Safety Advisor,6,6,28,3
3,2014-01-06 06:28:05,35,36.076217,24.874253,Vasco-Pais,Willem,Executive,Environmental Safety Advisor,6,6,28,5
4,2014-01-06 06:28:06,35,36.076214,24.874167,Vasco-Pais,Willem,Executive,Environmental Safety Advisor,6,6,28,6


In [21]:
preprocessor = ColumnTransformer(
    [
        ( 'dropper', 'drop', ['Timestamp'] ),
        ('encoder', OneHotEncoder(), ['LastName', 'FirstName', 'CurrentEmploymentType', 'CurrentEmploymentTitle'])
    ],
    remainder='passthrough'
)

pipe = Pipeline(
    [
        ('preprocessor', preprocessor),
        ('model', IsolationForest())
    ]
)

In [22]:
preds = pipe.fit_predict(locations)

In [23]:
pd.DataFrame(preds)[0].unique()

array([1])

In [26]:
buys = pd.read_csv('../CheckPoints/Buys_Clean.csv', parse_dates=['timestamp'], index_col=0)
buys.head()

,timestamp,location,price,FirstName,LastName,Is_Loyalty,day,hour,minute,second,cents,CarID,CurrentEmploymentType,CurrentEmploymentTitle
0,2014-01-06 07:28:00,Brew've Been Served,11.34,Edvard,Vann,True,6,7,28,0,34.0,34.0,Security,Perimeter Control
1,2014-01-06 07:34:00,Hallowed Grounds,12.22,Hideki,Cocinaro,True,6,7,34,0,22.0,12.0,Security,Site Control
2,2014-01-06 07:35:00,Brew've Been Served,8.33,Stenig,Fusil,True,6,7,35,0,33.0,20.0,Security,Building Control
3,2014-01-06 07:36:00,Hallowed Grounds,16.72,Birgitta,Frente,True,6,7,36,0,72.0,18.0,Engineering,Geologist
4,2014-01-06 07:37:00,Brew've Been Served,4.24,Sven,Flecha,True,6,7,37,0,24.0,17.0,Information Technology,IT Technician


In [30]:
preprocessor = ColumnTransformer(
    [
        ( 'dropper', 'drop', ['timestamp', 'cents', 'CarID'] ),
        ('encoder', OneHotEncoder(), ['location', 'LastName', 'FirstName', 'CurrentEmploymentType', 'CurrentEmploymentTitle'])
    ],
    remainder='passthrough'
)

pipe = Pipeline(
    [
        ('preprocessor', preprocessor),
        ('model', IsolationForest())
    ]
)

preds = pipe.fit_predict(buys)
pd.DataFrame(preds)[0].unique()

array([1])